In [1]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.filterwarnings('ignore')

In [2]:
actor = pd.read_csv('../data/actor.csv', encoding='latin1')
a=actor.copy()
category = pd.read_csv('../data/category.csv', encoding='latin1')
c=category.copy()
film = pd.read_csv('../data/film.csv', encoding='latin1')
f=film.copy()
i = pd.read_csv('../data/inventory.csv', encoding='latin1')
inventory=i.copy()
language = pd.read_csv('../data/language.csv', encoding='latin1')
l=language.copy()
old_HDD = pd.read_csv('../data/old_HDD.csv', encoding='latin1')
o=old_HDD.copy()
rental = pd.read_csv('../data/rental.csv', encoding='latin1')
r=rental.copy()

## Exploración archivos y limpieza previa.

In [3]:
#1º. Actores.

a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [4]:
a.describe()

,actor_id
count,200.000000
mean,100.500000
std,57.879185
min,1.000000
25%,50.750000
50%,100.500000
75%,150.250000
max,200.000000


In [5]:
a.isnull().sum() #No hay ningun valor nulo

actor_id       0
first_name     0
last_name      0
last_update    0
dtype: int64

In [6]:
a.columns #vamos a ponerlo bonito

Index(['actor_id', 'first_name', 'last_name', 'last_update'], dtype='object')

In [7]:
a.columns=a.columns.str.replace('_',' ').str.title()
a.columns

Index(['Actor Id', 'First Name', 'Last Name', 'Last Update'], dtype='object')

In [8]:
a['Last Update'].unique() #toda la columna es igual y no nos da ninguna información relevante, nos la cargamos 🆑

array(['2006-02-15 04:34:33'], dtype=object)

In [9]:
a.drop(columns=['Last Update'], inplace=True)
a.columns

Index(['Actor Id', 'First Name', 'Last Name'], dtype='object')

In [10]:
a.columns.duplicated().any() #sin duplicados

False

In [11]:
a[a.duplicated(subset=['First Name', 'Last Name'],keep=False)] # busco duplicados pero me centro solo en los nombes ya que los id son diferentes para cada uno,
                                                               # hay un actor repetido, vamos a quitarlo. El keep False lo saque del chat 🤫, porque si no
                                                               #lo ponía solo me mostraba el 109 con el nombre y no sabia de donde venía el duplicado.

,Actor Id,First Name,Last Name
100,101,SUSAN,DAVIS
109,110,SUSAN,DAVIS


In [12]:
a.drop([109],axis=0,inplace=True) # no me va a interferir porque no hay ninguna tabla que me relacione su id con nada, no peta 💣

In [13]:
a.reset_index(inplace=True,drop=True)
nuevos_id = pd.Series(range(1, len(a)+1)) # cambio los id para que no tener salto y reseteo index
a['Actor Id']=nuevos_id

a.iloc[107:112] # compruebo que no me pegue el salto, he hecho todo esto porque para el análisis me puede servir

,Actor Id,First Name,Last Name
107,108,WARREN,NOLTE
108,109,SYLVESTER,DERN
109,110,CAMERON,ZELLWEGER
110,111,RUSSELL,BACALL
111,112,MORGAN,HOPKINS


In [14]:
#2º) Categoría: 

c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  16 non-null     int64 
 1   name         16 non-null     object
 2   last_update  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


In [15]:
c.head()

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [16]:
c.describe()

,category_id
count,16.000000
mean,8.500000
std,4.760952
min,1.000000
25%,4.750000
50%,8.500000
75%,12.250000
max,16.000000


In [17]:
c.isnull().sum() #sin nulos


category_id    0
name           0
last_update    0
dtype: int64

In [18]:
c.columns=c.columns.str.replace('_',' ').str.title() #ponemos bonitas las columnas

In [19]:
c.drop(columns=['Last Update'], inplace=True) #quitamos las update, no me vale
c.columns

Index(['Category Id', 'Name'], dtype='object')

In [20]:
c.duplicated().sum() #sin duplicados

0

In [21]:
#3º) Film:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  special_features      1000 non-null   object 
 12  last_update           1000 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 101.7+ KB


In [22]:
f.describe()

,film_id,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost
count,1000.000000,1000.0,1000.0,0.0,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,2006.0,1.0,NaN,4.985000,2.980000,115.272000,19.984000
std,288.819436,0.0,0.0,NaN,1.411654,1.646393,40.426332,6.050833
min,1.000000,2006.0,1.0,NaN,3.000000,0.990000,46.000000,9.990000
25%,250.750000,2006.0,1.0,NaN,4.000000,0.990000,80.000000,14.990000
50%,500.500000,2006.0,1.0,NaN,5.000000,2.990000,114.000000,19.990000
75%,750.250000,2006.0,1.0,NaN,6.000000,4.990000,149.250000,24.990000
max,1000.000000,2006.0,1.0,NaN,7.000000,4.990000,185.000000,29.990000


In [23]:
f.columns=f.columns.str.replace('_',' ').str.title()

In [24]:
f.isna().sum() 

Film Id                    0
Title                      0
Description                0
Release Year               0
Language Id                0
Original Language Id    1000
Rental Duration            0
Rental Rate                0
Length                     0
Replacement Cost           0
Rating                     0
Special Features           0
Last Update                0
dtype: int64

In [25]:
f.drop(columns=['Original Language Id','Last Update'], inplace=True) #tiene una columna llena de nulos, la quitamos junto con Last
f.columns

Index(['Film Id', 'Title', 'Description', 'Release Year', 'Language Id',
       'Rental Duration', 'Rental Rate', 'Length', 'Replacement Cost',
       'Rating', 'Special Features'],
      dtype='object')

In [26]:
#4º) Inventory:

i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   inventory_id  1000 non-null   int64 
 1   film_id       1000 non-null   int64 
 2   store_id      1000 non-null   int64 
 3   last_update   1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [27]:
i.describe()

,inventory_id,film_id,store_id
count,1000.000000,1000.000000,1000.000000
mean,500.500000,109.866000,1.497000
std,288.819436,63.862042,0.500241
min,1.000000,1.000000,1.000000
25%,250.750000,56.000000,1.000000
50%,500.500000,111.500000,1.000000
75%,750.250000,164.000000,2.000000
max,1000.000000,223.000000,2.000000


In [28]:
i.columns=i.columns.str.replace('_',' ').str.title()

In [29]:
i.head() 

,Inventory Id,Film Id,Store Id,Last Update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [30]:
i.isna().sum()

Inventory Id    0
Film Id         0
Store Id        0
Last Update     0
dtype: int64

In [31]:
i.duplicated().sum()

0

In [32]:
#5º) Language:

l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  6 non-null      int64 
 1   name         6 non-null      object
 2   last_update  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 272.0+ bytes


In [33]:
l.describe()

,language_id
count,6.000000
mean,3.500000
std,1.870829
min,1.000000
25%,2.250000
50%,3.500000
75%,4.750000
max,6.000000


In [34]:
l.isna().sum()

language_id    0
name           0
last_update    0
dtype: int64

In [35]:
l.duplicated().sum

<bound method Series.sum of 0    False
1    False
2    False
3    False
4    False
5    False
dtype: bool>

In [36]:
l.columns=l.columns.str.replace('_',' ').str.title()

In [37]:
#6º) old_HDD:
o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [38]:
o.describe()

,release_year,category_id
count,1000.0,1000.000000
mean,2006.0,8.355000
std,0.0,4.726872
min,2006.0,1.000000
25%,2006.0,4.000000
50%,2006.0,8.000000
75%,2006.0,13.000000
max,2006.0,16.000000


In [39]:
o.isna().sum()

first_name      0
last_name       0
title           0
release_year    0
category_id     0
dtype: int64

In [40]:
o.columns=o.columns.str.replace('_',' ').str.title()

In [41]:
o[o.duplicated(subset=['Title', 'First Name', 'Last Name'], keep=False)] # ningun duplicado, los he pasado así por si había algun actor repetido dentro de una 
                                                                         # misma peli


,First Name,Last Name,Title,Release Year,Category Id


In [42]:
#7º) Rentail:

r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
 6   last_update   1000 non-null   object
dtypes: int64(4), object(3)
memory usage: 54.8+ KB


In [43]:
r.describe()

,rental_id,inventory_id,customer_id,staff_id
count,1000.00000,1000.000000,1000.000000,1000.000000
mean,501.18000,2258.179000,296.408000,1.521000
std,289.19665,1314.667454,172.509319,0.499809
min,1.00000,2.000000,1.000000,1.000000
25%,250.75000,1157.500000,150.000000,1.000000
50%,501.50000,2244.500000,296.000000,2.000000
75%,751.25000,3371.750000,445.250000,2.000000
max,1001.00000,4581.000000,597.000000,2.000000


In [44]:
r.columns=r.columns.str.replace('_',' ').str.title()

In [45]:
r.isna().sum()

Rental Id       0
Rental Date     0
Inventory Id    0
Customer Id     0
Return Date     0
Staff Id        0
Last Update     0
dtype: int64

In [46]:
r.duplicated().sum()

0

In [47]:
o[o.duplicated(subset=['Title', 'First Name', 'Last Name'], keep=False)] # lo hacemos así para comprobar que no se repita el mismo actor en la misma peli

,First Name,Last Name,Title,Release Year,Category Id
